In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now().strftime("%d.%m.%Y")} / Tatu Erkinjuntti')

Päivitetty 08.11.2024 / Tatu Erkinjuntti


# Tehtävä 2.



## Osa 1 - aikasarjat


Valitse kahden pörssiosakkeen kurssihistoriat, ei kuitenkaan esimerkkimuistioissa esiintyviä Elisaa tai Teliaa (voi myös käyttää raaka-aineiden tai virtuaalivaluuttojen hintahistorioita) ja analysoi niitä opituilla menetelmillä. Lisää mukaan mahdollisimman paljon omia kommentteja ja selitystä.

Osakkeiden kurssihistorioita ja raaka-aineiden/valuuttojen hintahistorioita saat Yahoo Finance -palvelusta:

1) avaa https://finance.yahoo.com/

2) Etsi (search) haluamasi yritys esim NOKIA.HE

3) Klikkaa "Historical Data

4) Määritä päivämäärät haulle ja sitten "download"


## Osa 2 - aikasarjaennustaminen


Valitse yksi seuraavista aikasarjoista tai käytä jotain omaa aikasarjaa ja analysoi kommentoiden opituilla menetelmillä.

### Aikasarjoja

Seuraavissa aikasarjoissa on kaikissa sekä trendi että kausivaihtelu.

#### Ilman CO2-pitoisuuksia kuukausittain. 
Siististi käyttäytyvä aikasarja, jolle saa laskettua tarkkoja ennusteita.

Lähde: https://www.esrl.noaa.gov/gmd/webdata/ccgg/trends/co2/co2_mm_mlo.txt

Aukenee säädylliseen dataframeen suoraan lähteestä seuraavan esimerkin mukaisesti.

#### Lentomatkustajien lukumääriä AirPassengers.csv. 

Tästä aikasarjasta löytyy netistä paljon laskettuja esimerkkejä eri ennustusmenetelmille.

Lähde: https://www.kaggle.com/rakannimer/air-passengers

tai täällä


Aukenee säädylliseen dataframeen seuraavasti:

df=pd.read_csv('AirPassengers.csv')
df.index=pd.to_datetime(df['Month'],format='%Y-%m')
df=df.drop('Month',axis=1)
df.head()

#### Oluen tuotantomääriä beer.csv

Lähde: https://www.kaggle.com/shenba/time-series-datasets

löytyy myös täältä

Aukenee säädylliseen dataframeen seuraavasti:

df=pd.read_csv('beer.csv')
df.index=pd.to_datetime(df['Month'],format='%Y-%m')
df=df.drop('Month',axis=1)
df.head()

#### Sähkön tuotantoa Electric_Production.csv (löytyy data-paketista)

Lähde: https://www.kaggle.com/shenba/time-series-datasets

löytyy myös täältä

Aukenee säädylliseen dataframeen seuraavasti:

df=pd.read_csv('Electric_Production.csv')
df.index=pd.to_datetime(df['DATE'],format='%m/%d/%Y')
df=df.drop('DATE',axis=1)
df.head()




## Tehtävän alustus

Haetaan tarvittavat kirjastot ja määritellään käytetyt data-aineistot. Itse teen tämän aina kerralla ohjelman alussa, näin yhdellä silmäyksellä näkee ohjelman riippuvaisuudet. Jos käytössä olisi pysyviä muuttujia, tulisi ne myös alustaa samalla.  

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

import yfinance as yf

from scipy.stats import pearsonr
from scipy.stats import ttest_ind

# Kaavioiden prosenttiakselin muotoiluun
from matplotlib.ticker import PercentFormatter
ticks = PercentFormatter(xmax=100, decimals=0, symbol=' %')

sns.set_style('dark')

electric_procuction = '../Data/Electric_Production.csv'

## Osa 1, aikasarjat

Valitsin tähän tehtävään kaksi kurssihistoriaa, Nvidia (NVDA) sekä Bitcoin (BTC-USD). 
Syy tähän on oikeastaan se että molemmilla on viimeisen 10 vuoden aikana ollut sangen korkealentoinen kurssikehitys.

- Nvidian kurssihistoria perustuu Nasdaq:ssa käytyyn kauppaan.
- Bitcoinin kurssihistoria perustuu sen suhteeseen Yhdysvaltain Dollarin arvoon.

Aloitetaan hakemalla data-aineisto.

In [3]:
# HUOM! koska aikaisemmin mainittiin 10 vuotta, niin ulotetaan data-aineisto 10 vuoden päähän.
ndivia_d = yf.download('NVDA', start='2014-1-1')
bitcoin_d = yf.download('BTC-USD', start='2014-1-1')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Aloitetaan tutkimalla Nvidian data-aineisto

In [4]:
ndivia_d

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,,
2014-01-02 00:00:00+00:00,0.373992,0.396500,0.399500,0.393000,0.398000,260092000
2014-01-03 00:00:00+00:00,0.369512,0.391750,0.398000,0.390500,0.397250,259332000
2014-01-06 00:00:00+00:00,0.374464,0.397000,0.400000,0.392000,0.395750,409492000
2014-01-07 00:00:00+00:00,0.380595,0.403500,0.405000,0.398250,0.401000,333288000
2014-01-08 00:00:00+00:00,0.385782,0.409000,0.411000,0.403500,0.405000,308192000
...,...,...,...,...,...,...
2024-11-01 00:00:00+00:00,135.399994,135.399994,137.309998,134.570007,134.699997,207127800
2024-11-04 00:00:00+00:00,136.050003,136.050003,138.960007,135.570007,137.210007,187528200


Katsotaan nyt miltä Bitcoinin data-aineisto näyttää.

In [5]:
bitcoin_d

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,,
2014-09-17 00:00:00+00:00,457.334015,457.334015,468.174011,452.421997,465.864014,21056800
2014-09-18 00:00:00+00:00,424.440002,424.440002,456.859985,413.104004,456.859985,34483200
2014-09-19 00:00:00+00:00,394.795990,394.795990,427.834991,384.532013,424.102997,37919700
2014-09-20 00:00:00+00:00,408.903992,408.903992,423.295990,389.882996,394.673004,36863600
2014-09-21 00:00:00+00:00,398.821014,398.821014,412.425995,393.181000,408.084991,26580100
...,...,...,...,...,...,...
2024-11-03 00:00:00+00:00,68741.117188,68741.117188,69361.656250,67482.523438,69296.382812,34868307655
2024-11-04 00:00:00+00:00,67811.507812,67811.507812,69433.179688,66803.648438,68742.132812,41184819348


Huomaamme data-aineiston riveistä, että koska valittu aineisto yltää yli kymmenen vuoden päähän, on meillä molemmissa tapauksissa sopivan suuri aineisto pohja.